# TensorTrade - Renderers and Plotly Visualization Chart
## Data Loading Function

In [17]:
# ipywidgets is required to run Plotly in Jupyter Notebook.
# Uncomment and run the following line to install it if required.
!pip install -r requirements.txt -qU --no-cache-dir
!pip check ipywidgets &>/dev/null || pip install ipywidgets ta -qU --no-cache-dir

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [18]:
import ta

import pandas as pd

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC
from tensortrade.oms.wallets import Wallet, Portfolio

%matplotlib inline

In [19]:
cdd = CryptoDataDownload()
df = pd.concat([
    cdd.fetch("Bitfinex", "USD", "BTC", "1h").add_prefix("BTC:")
], axis=1)
df.head()

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,1526364000,8723.8,8793.0,8714.9,8739.0,8988053.53
1,2018-05-15 07:00:00,1526367600,8739.0,8754.8,8719.3,8743.0,2288904.12
2,2018-05-15 08:00:00,1526371200,8743.0,8743.1,8653.2,8723.7,8891773.14
3,2018-05-15 09:00:00,1526374800,8723.7,8737.8,8701.2,8708.1,2054868.28
4,2018-05-15 10:00:00,1526378400,8708.1,8855.7,8695.8,8784.4,17309722.58


## Data Preparation
### Create the dataset features

In [20]:
dataset = ta.add_all_ta_features(df, 'BTC:open', 'BTC:high', 'BTC:low', 'BTC:close', 'BTC:volume', fillna=True)
dataset.head(3)

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,volume_adi,volume_obv,volume_cmf,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2018-05-15 06:00:00,1526364000,8723.8,8793.0,8714.9,8739.0,8988053.53,-3.441009e+06,8988053.53,-0.382843,...,-69.142125,0.0,8739.000000,0.0,0.000000,0.000000,0.000000,-25.322299,0.000000,0.000000
1,2018-05-15 07:00:00,1526367600,8739.0,8754.8,8719.3,8743.0,2288904.12,-2.673743e+06,11276957.65,-0.237098,...,-64.020487,0.0,8740.774500,0.0,-6.293175,-1.258635,-5.034540,0.045772,0.045761,0.045772
2,2018-05-15 08:00:00,1526371200,8743.0,8743.1,8653.2,8723.7,8891773.14,2.380424e+06,2385184.51,0.118025,...,-49.570815,0.0,8733.276121,0.0,-4.932072,-1.993322,-2.938749,-0.220748,-0.220992,-0.175077


### Create Chart Price History Data
Note: It is recommended to create the chart data *after* creating and cleaning the dataset to ensure one-to-one mapping between the historical prices data and the dataset.

In [21]:
price_history = dataset[['BTC:date', 'BTC:open', 'BTC:high', 'BTC:low', 'BTC:close', 'BTC:volume']]  # chart data
display(price_history.head(3))

dataset.drop(columns=['BTC:date', 'BTC:open', 'BTC:high', 'BTC:low', 'BTC:close', 'BTC:volume'], inplace=True)

,BTC:date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,8723.8,8793.0,8714.9,8739.0,8988053.53
1,2018-05-15 07:00:00,8739.0,8754.8,8719.3,8743.0,2288904.12
2,2018-05-15 08:00:00,8743.0,8743.1,8653.2,8723.7,8891773.14


## Setup Trading Environment
### Create Data Feeds

In [22]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(price_history['BTC:close'].tolist(), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
])

with NameSpace("bitfinex"):
    streams = [Stream.source(dataset[c].tolist(), dtype="float").rename(c) for c in dataset.columns]

feed = DataFeed(streams)
feed.next()

{'bitfinex:/BTC:unix': 1526364000,
 'bitfinex:/volume_adi': -3441008.969871901,
 'bitfinex:/volume_obv': 8988053.53,
 'bitfinex:/volume_cmf': -0.38284250960306654,
 'bitfinex:/volume_fi': 0.0,
 'bitfinex:/volume_mfi': 50.0,
 'bitfinex:/volume_em': 0.0,
 'bitfinex:/volume_sma_em': 0.0,
 'bitfinex:/volume_vpt': -2278209.0941296634,
 'bitfinex:/volume_nvi': 1000.0,
 'bitfinex:/volume_vwap': 8748.966666666667,
 'bitfinex:/volatility_atr': 0.0,
 'bitfinex:/volatility_bbm': 8739.0,
 'bitfinex:/volatility_bbh': 8739.0,
 'bitfinex:/volatility_bbl': 8739.0,
 'bitfinex:/volatility_bbw': 0.0,
 'bitfinex:/volatility_bbp': 0.0,
 'bitfinex:/volatility_bbhi': 0.0,
 'bitfinex:/volatility_bbli': 0.0,
 'bitfinex:/volatility_kcc': 8748.966666666667,
 'bitfinex:/volatility_kch': 8827.066666666668,
 'bitfinex:/volatility_kcl': 8670.866666666667,
 'bitfinex:/volatility_kcw': 1.7853536989130225,
 'bitfinex:/volatility_kcp': 0.4361929150661517,
 'bitfinex:/volatility_kchi': 0.0,
 'bitfinex:/volatility_kcli': 

### Trading Environment Renderers
A renderer is a channel for the trading environment to output its current state. One or more renderers can be attached to the environment at the same time. For example, you can let the environment draw a chart and log to a file at the same time.

Notice that while all renderers can technically be used together, you need to select the best combination to avoid undesired results. For example, PlotlyTradingChart can work well with FileLogger but may not display well with ScreenLogger.

Renderer can be set by name (string) or class, single or list. Available renderers are:
* `'screenlog'` or `ScreenLogger`: Shows results on the screen.
* `'filelog'` or `FileLogger`: Logs results to a file.
* `'plotly'` or `PlotlyTradingChart`: A trading chart based on Plotly.

#### Examples:

* renderers = 'screenlog' (default)
* renderers = ['screenlog', 'filelog']
* renderers = ScreenLogger()
* renderers = ['screenlog', `FileLogger()`]
* renderers = [`FileLogger(filename='example.log')`]

Renderers can also be created and configured first then attached to the environment as seen in a following example.

### Trading Environment with a Single Renderer

In [23]:
import tensortrade.env.default as default

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer="screen-log",  # ScreenLogger used with default settings
    window_size=20
)

In [24]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)
agent.train(n_episodes=2, n_steps=200, render_interval=10)

====      AGENT ID: 0bc064c4-6da1-42eb-8462-6a27c45d95d6      ====
[2021-04-02 1:07:31 PM] Episode: 1/2 Step: 131/200
[2021-04-02 1:07:32 PM] Episode: 1/2 Step: 141/200
[2021-04-02 1:07:33 PM] Episode: 1/2 Step: 151/200
[2021-04-02 1:07:34 PM] Episode: 1/2 Step: 161/200
[2021-04-02 1:07:35 PM] Episode: 1/2 Step: 171/200
[2021-04-02 1:07:36 PM] Episode: 1/2 Step: 181/200
[2021-04-02 1:07:37 PM] Episode: 1/2 Step: 191/200
[2021-04-02 1:07:38 PM] Episode: 1/2 Step: 201/200
[2021-04-02 1:07:39 PM] Episode: 2/2 Step: 11/200
[2021-04-02 1:07:40 PM] Episode: 2/2 Step: 21/200
[2021-04-02 1:07:41 PM] Episode: 2/2 Step: 31/200
[2021-04-02 1:07:42 PM] Episode: 2/2 Step: 41/200
[2021-04-02 1:07:43 PM] Episode: 2/2 Step: 51/200
[2021-04-02 1:07:44 PM] Episode: 2/2 Step: 61/200
[2021-04-02 1:07:45 PM] Episode: 2/2 Step: 71/200
[2021-04-02 1:07:46 PM] Episode: 2/2 Step: 81/200
[2021-04-02 1:07:47 PM] Episode: 2/2 Step: 91/200
[2021-04-02 1:07:47 PM] Episode: 2/2 Step: 101/200
[2021-04-02 1:07:48 PM] 

-996367.7077659033

### Environment with Multiple Renderers
Create PlotlyTradingChart and FileLogger renderers. Configuring renderers is optional as they can be used with their default settings.

In [25]:
from tensortrade.env.default.renderers import PlotlyTradingChart, FileLogger

chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format="html",  # save the chart to an HTML file
    auto_open_html=True,  # open the saved HTML chart in a new browser tab
)

file_logger = FileLogger(
    filename="example.log",  # omit or None for automatic file name
    path="training_logs"  # create a new directory if doesn't exist, None for no directory
)

### Environement with Multiple Renderers

With the plotly renderer you must provide an parameter called `renderer_feed`. This is a `DataFeed` instance that provides all the information that is required by a renderer to function.

In [26]:
renderer_feed = DataFeed([
    Stream.source(price_history[c].tolist(), dtype="float").rename(c) for c in price_history]
)

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    window_size=20,
    renderer_feed=renderer_feed,
    renderers=[
        chart_renderer, 
        file_logger
    ]
)

## Setup and Train DQN Agent
The green and red arrows shown on the chart represent buy and sell trades respectively. The head of each arrow falls at the trade execution price.

In [27]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)

# Set render_interval to None to render at episode ends only
agent.train(n_episodes=2, n_steps=200, render_interval=10)

====      AGENT ID: 9ca0a0c7-d007-4c6b-8c85-c4bd6e4d3d56      ====


-1026204.5763763315

## Direct Performance and Net Worth Plotting
Alternatively, the final performance and net worth can be displayed using pandas via Matplotlib.

In [28]:
%matplotlib inline

portfolio.performance.plot()

AttributeError: 'collections.OrderedDict' object has no attribute 'plot'

In [ ]:
portfolio.performance.net_worth.plot()